# OpenCV-Tutorials 3- Image Processing Techniques

The value of pixels will be the features for a Deep learning model. So it is important to give the model only the important features. So we need to remove noise, transform the image, smooth the image, etc to get a better result.
OpenCV provides below techniques to process the pixels of the image:

__Contents:__
<ol>
    <li>Thresholding</li>
    <li>Color Filters</li>
    <li>Smoothing-Denoising transformations(To remove noise)</li>
    <li>Blurring (To remove noise)</li>
</ol>
    


In [42]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [49]:
image = cv2.imread('images/dogs/bookpage.jpg')

In [50]:
def show_img(img):
    cv2.startWindowThread()
    cv2.namedWindow("preview")
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Part-1
## 1.Thresholding:

<br>There are 2 types of thresholding:
<br>1.Global Thresholding
<br>2.Adaptive Thresholding
<br>

1.__Global Thresholding__:
We select a value for threshold. If the pixel value is greater than some threshold, we change the pixel value to something else

2.__Adaptive Thresholding__:
The algorithm automatically selects the threshold as the image could need different thresholds at different places


### Global Thresholding:
__cv2.threshold(image,threshold_value,upgrade_value,type_of_threshold)__ returns the image_array with values thresholded

<br>types of threshold:
<br>cv2.THRESH_BINARY
<br>cv2.THRESH_BINARY_INV
<br>cv2.THRESH_TRUNC
<br>cv2.THRESH_TOZERO
<br>cv2.THRESH_TOZERO_INV
<br>
We can add cv.THRESH_OTSU to these thresholds to select threshold automatically

for detailed explanation, please visit https://docs.opencv.org/3.4.3/d7/d4d/tutorial_py_thresholding.html

In [51]:
image = cv2.imread('images/sample/noise.jpg',0)
show_img(image)

In [ ]:
(_,new_image1) = cv2.threshold(image,12,255,cv2.THRESH_BINARY)
show_img(new_image1)

In [ ]:
(_,new_image1) = cv2.threshold(image,12,255,cv2.THRESH_BINARY_INV)
show_img(new_image1)

In [ ]:
(_,new_image1) = cv2.threshold(image,12,255,cv2.THRESH_TRUNC)
show_img(new_image1)

In [ ]:
(_,new_image1) = cv2.threshold(image,12,255,cv2.THRESH_TOZERO)
show_img(new_image1)

In [ ]:
(_,new_image1) = cv2.threshold(image,12,255,cv2.THRESH_TOZERO_INV)
show_img(new_image1)

#### Lets add cv2.THRESH_OTSU to threshold_type to select threshold automatically

In [ ]:
for threshold_type in [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC,cv2.THRESH_TOZERO,cv2.THRESH_TOZERO_INV]:
    show_img(cv2.threshold(image,12,255,threshold_type+cv2.THRESH_OTSU)[1])

### Adaptive Thresholding:

__adaptiveThreshold(src, maxValue, adaptiveMethod, thresholdType, blockSize, C)__ 

adaptiveMethods:
<ol>
    <li>cv2.ADAPTIVE_THRESH_MEAN_C</li>
    <li>cv2.ADAPTIVE_THRESH_GAUSSIAN_C</li>
</ol>

thresholdTypes can be:
<ol>
    <li>cv2.THRESH_BINARY</li>
    <li>cv2.THRESH_BINARY_INV</li>
</ol>

For details, please visit https://docs.opencv.org/3.4.3/d7/d4d/tutorial_py_thresholding.html

In [ ]:
new_image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
show_img(new_image)

#### Lets run all the possible combinations

In [ ]:
for adaptive_method in [cv2.ADAPTIVE_THRESH_MEAN_C,cv2.ADAPTIVE_THRESH_GAUSSIAN_C]:
    for threshold_type in [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV]:
        show_img(cv2.adaptiveThreshold(image,255,adaptive_method,threshold_type,11,2))

# Part-2
## Color Filters:

__cv2.inRange(hsv,lower_range_hsv,upper_range_hsv)__ retruns only pixels which are in range from lower range hsv value to upper range hsv value
<br>


to convert bgr to hsv values i.e.to calculate lower and upper bound hsv values use external website like
https://www.rapidtables.com/convert/color/rgb-to-hsv.html

In [4]:
def color_filter(img,lower_range_hsv,upper_range_hsv):
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    return(cv2.inRange(hsv,lower_range_hsv,upper_range_hsv))

In [5]:
def color_filter_fill(img,lower_range_hsv,upper_range_hsv):
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv,lower_range_hsv,upper_range_hsv)
    return(cv2.bitwise_and(img,img,mask=mask))

In [6]:
def hsv2bgr(hsv):
    return(cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR))
print(hsv2bgr(np.uint8([[[180,255,255]]])))

[[[  0   0 255]]]


In [9]:
capture = cv2.VideoCapture(1)
while True:
    _,frame = capture.read()
    cv2.imshow('original',frame)
    cv2.imshow('filtered_mask',color_filter(frame,np.array([150,150,50]),np.array([180,255,180])))#filter red
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows() 


#### Lets fill the white area i.e. red in original image, white in masked image with original pixels.

In [16]:
capture = cv2.VideoCapture(1)
while True:
    _,frame = capture.read()
    cv2.imshow('original',frame)
    cv2.imshow('filtered_mask',color_filter(frame,np.array([150,150,50]),np.array([180,255,180])))#filter red
    cv2.imshow('filtered_mask',color_filter_fill(frame,np.array([150,150,50]),np.array([180,255,180])))#filter red
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows() 

<img src="images/color_filter_red.png" style="width:100px;height:100;">

# Part-3
## Denoising(smoothing) the image

<ol>
    <li>Erosion</li>
    <li>Dilation</li>
    <li>combinations of erosion,dilation and original image as Morphological transformations</li>
</ol>

In [113]:
image = cv2.imread('images/sample/noise.jpg')

### Erosion:
It erodes away the boundaries of foreground object (Always try to keep foreground in white). So what it does? The kernel slides through the image (as in 2D convolution). A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the kernel is 1, otherwise it is eroded (made to zero).

In [106]:
kernel = np.ones((5,5),np.uint8)
eroded_image = cv2.erode(image,kernel,iterations=1)

In [107]:
show_img(image)
show_img(eroded_image)

### Dilation:
It is just opposite of erosion. Here, a pixel element is '1' if atleast one pixel under the kernel is '1'. So it increases the white region in the image or size of foreground object increases. Normally, in cases like noise removal, erosion is followed by dilation. Because, erosion removes white noises, but it also shrinks our object. So we dilate it. Since noise is gone, they won't come back, but our object area increases. It is also useful in joining broken parts of an object.

In [108]:
dilated_image = cv2.dilate(image,kernel,iterations=1)

In [109]:
show_img(image)
show_img(dilated_image)

### Morphologic Transformations

__cv2.morphologyEx(image, morphology_type , kernel)__ returns array of transformed image

morphology_type can be :
<ol>
    <li>cv2.MORPH_OPEN</li>
    <li>cv2.MORPH_CLOSE</li>
    <li>cv2.MORPH_GRADIENT</li>
    <li>cv2.MORPH_TOPHAT</li>
    <li>cv2.MORPH_BLACKHAT</li>
</ol>
<br>
kernel can be :
<ol>
    <li>__Rectangular kernel__: cv2.getStructuringElement(cv2.MORPH_RECT,(rows,colums))</li>
    <li>__Elliptical Kernel__: cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(rows,columns))</li>
    <li>__Cross-shaped Kernel__: cv2.getStructuringElement(cv2.MORPH_CROSS,(rows,columns))</li>
    <li>__random matrix shape less than image__: np.ones((5,5),np.uint8)/225
</ol>
<br>

When to apply morphology_type: 
<br>__cv2.MORPH_OPEN__ : Opening is just another name of erosion followed by dilation. It is useful in removing noise
<br>__cv2.MORPH_CLOSE__ : Closing is reverse of Opening, Dilation followed by Erosion. It is useful in closing small holes inside the foreground objects, or small black points on the object.
<br>__cv2.MORPH_GRADIENT__: It is the difference between dilation and erosion of an image.
<br>__cv2.MORPH_TOPHAT__ : It is the difference between input image and Opening of the image
<br>__cv2.MORPH_BLACKHAT__ : It is the difference between the closing of the input image and input image.


In [110]:
morphologic_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

In [111]:
show_img(image)
show_img(morphologic_image)

In [128]:
for morphological_type in [cv2.MORPH_OPEN,cv2.MORPH_CLOSE,cv2.MORPH_GRADIENT,cv2.MORPH_TOPHAT,cv2.MORPH_BLACKHAT]:
    for kernel in [cv2.getStructuringElement(cv2.MORPH_RECT,(5,5)),cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)),cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5)),np.ones((5,5),np.uint8)/225]:
        show_img(image)
        show_img(cv2.morphologyEx(image, morphological_type, kernel))

# Part-4
## Blurring

<ol>
    <li>filter2d</li>
    <li>

In [114]:
image = cv2.imread('images/sample/noise.jpg')

In [121]:
kernel = np.ones((5,5),np.uint8)/255
filtered_image = cv2.filter2D(image,-1,kernel)
show_img(filtered_image)

In [126]:
show_img(image)
show_img(cv2.blur(image,(5,5)))

In [127]:
show_img(image)
show_img(cv2.GaussianBlur(image,(5,5),0))